## Setting up HDDM

In [1]:
import pandas as pd
import hddm

rawData = hddm.load_csv('study1nonadaptive.csv')
data = rawData[rawData['sure'] == 0]
data['choice'] = data['choice'] ^ 1
data['loss'] = data['loss'] * (-1)
data.rename(columns = {'id':'subj_idx', 'choice':'response', 'win':'gain'}, inplace = True)

/anaconda3/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python2.7/site-packages/pandas/core/frame.py:40

In [2]:
data.head()

,subj_idx,trial,rt,response,gain,loss,sure
1,1,2,3.790270,1,12.0,-7.50,0.0
3,1,4,2.003780,1,10.0,-15.00,0.0
4,1,5,3.993464,0,2.0,-3.00,0.0
7,1,8,4.908444,0,10.0,-12.50,0.0
8,1,9,1.064644,1,12.0,-3.75,0.0


In [3]:
m = hddm.models.HDDMRegressor(data, ['v ~ I(gain + loss)'], include=('z'), group_only_regressors=False)

Adding these covariates:
['v_Intercept', 'v_I(gain + loss)']


In [4]:
m.find_starting_values()

/anaconda3/lib/python2.7/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/anaconda3/lib/python2.7/site-packages/scipy/optimize/optimize.py:1935: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1


In [5]:
import os
modelName = 'no_loss_aversion'
if not os.path.exists('savedModels/{}'.format(modelName)):
    os.makedirs('savedModels/{}'.format(modelName))

m.sample(5000, burn=1000, dbname='savedModels/{}/db_{}'.format(modelName, modelName), db='pickle')

 [-----------------100%-----------------] 5001 of 5000 complete in 10633.4 sec

In [6]:
m.save('savedModels/{}/{}'.format(modelName, modelName))

## Model parameters

In [7]:
m.print_stats()
m.print_stats(fname='savedModels/{}/{}_stats.txt'.format(modelName, modelName))

                                mean         std         2.5q          25q         50q         75q       97.5q       mc err
a                            2.67978    0.104069      2.48334      2.60901     2.67896     2.74978     2.88583   0.00188453
a_std                       0.628355   0.0815691      0.49109     0.570158    0.622018     0.67797    0.810595   0.00196865
a_subj.1                     2.38663    0.106232      2.18229      2.31486     2.38388     2.45599      2.6052   0.00292031
a_subj.2                      2.0836   0.0740074      1.94264      2.03321     2.08115     2.13288     2.23533   0.00181289
a_subj.3                     2.86593     0.10722      2.66568      2.79387     2.86126     2.93582     3.08739   0.00247084
a_subj.4                      2.0175   0.0878978      1.85085      1.95703     2.01312     2.07527      2.1986   0.00210503
a_subj.5                     2.27123   0.0823099      2.11679      2.21511     2.26808     2.32496     2.44274    0.0018365
a_subj.6

In [8]:
modelParamsDF = m.nodes_db
modelParamsDF.to_csv('savedModels/{}/{}_nodes.csv'.format(modelName, modelName))

## Simulate data

In [9]:
ppc_data = hddm.utils.post_pred_gen(m, samples=50, progress_bar=True)

 [------------------107%------------------] 42 of 39 complete in 10645.0 sec

In [10]:
import numpy as np
dataArray = np.array(data)

def extractStakesFromParticipantIndex(participantIndex):
    participantData = dataArray[dataArray[:, 0] == participantIndex][:]
    participantStakes = participantData[:, 4:6]
    return participantStakes

In [11]:
import numpy as np

overallData = np.zeros((1, 5))

for (node, sample), sim_data in ppc_data.groupby(level=(0, 1)):
    print("-------")
    print("node: ", node)
    sim_data_array = np.asarray(sim_data)
    numTrials = np.shape(sim_data_array)[0]
    print("number of trials: ", numTrials)
    participantIndex = int(filter(str.isdigit, node))
    participantStakes = extractStakesFromParticipantIndex(participantIndex)
    participantIndexArray = np.full((numTrials, 1), participantIndex)
    responsesArray = np.zeros((numTrials, 1))
    responsesArray[sim_data_array[:, 0] > 0] = 1
    reactionTimesArray = np.abs(sim_data_array)
    sim_data_modified = np.hstack((participantIndexArray, responsesArray, reactionTimesArray, participantStakes))
    overallData = np.vstack((overallData, sim_data_modified))
    
print(np.shape(overallData))

-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('

-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.19')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node

...

-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.5')
('

-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('

In [12]:
import pandas as pd

import os
modelName = 'no_loss_aversion'
if not os.path.exists('simulatedData/{}'.format(modelName)):
    os.makedirs('simulatedData/{}'.format(modelName))

df = pd.DataFrame(overallData[1:, :])
df.columns = ['subj_idx', 'response', 'rt', 'gain', 'loss']
df.to_csv('simulatedData/{}/{}.csv'.format(modelName, modelName))
# np.savetxt("simulatedData.csv", overallData, delimiter=",", fmt='%.5f')